# Main Notebook

File to perform experiments

## Imports

In [1]:
import os
import numpy as np
import gpflow
import matplotlib.pyplot as plt
import pandas as pd
import time
from tqdm import tqdm
import tensorflow as tf

from models.GaussianProcess import GaussianProcess
from acquisition_functions.UseMO import usemo_acq
from acquisition_functions.MES import mes_acq, basic_mes_acq
from acquisition_functions.PESMO import pesmo_acq
from acquisition_functions.MESMO import mesmo_acq
from acquisition_functions.SingleObjective import simulated_mes_bins
from arguments.arguments import MainArguments


from MOObenchmark import MOOackley, MOOexponential, MOOquadratic_ackley, MOOnnHeartNeuronsProbF1
from utils.calc_pareto import get_pareto_undominated_by, getSetfromFront

from models.MOOEvaluationProblem import MOOEvaluationProblem

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_termination
from pymoo.optimize import minimize

2023-06-18 14:53:14.436974: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-18 14:53:14.501227: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-18 14:53:14.501895: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 14:53:15.151338: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Algorithm Arguments

In [2]:
d = 5 # input dimensions
    
seed = 0
np.random.seed(seed)

## iterations of the main algorithm
total_iter = 40
## iterations taken randomly at the beggining
initial_iter = 1

## bound of the inputs definition
### with the current implementations all the input dimensions must have the same bounds, 
### but this can be changed
lower_bound = 10
upper_bound = 1000

lowerBounds = [lower_bound]*d
upperBounds = [upper_bound]*d


## Evaluation

In [3]:
## Evaluation function definition
O = 1
C = 0

def evaluation(x):
    return MOOnnHeartNeuronsProbF1(x)

This cell below gets the real pareto in two ways: with a grid and with a cheap optimizer (genetic algorithm) and plots both of them. These results are used in test problems as the actual pareto front. Although, to obtain these paretos many evaluations of the function are needed, so it has no sense running it in a costly black box problem.

In addition this is only prepared to plot a d=1, O=2 problem. We could visualize any problem, but the visualization get a lot more abstract and more plots are needed. If this is interesting I could do it.

Therefore I set the code below as text.

N = 1_001
X = np.linspace(lower_bound,upper_bound,N)
Z = np.zeros((N,2))

problem = MOOEvaluationProblem(evaluation, O=O, d=d, lowerBounds=lower_bound, upperBounds=upper_bound)
algorithm = NSGA2()
res = minimize( problem, 
                algorithm,
                termination = get_termination("n_gen",100))

real_pareto = res.F[np.argsort(res.F[:,0])]

for i in range(N):
    Z[i]=evaluation(X[i])

fig, axs = plt.subplots(1,3, figsize=(15,4))

axs[0].plot(X, Z[:,0], 'b')
axs[0].plot(X, Z[:,1], 'k')
axs[0].plot(res.X, res.F[:,0], 'xr', markersize=5)
axs[0].plot(res.X, res.F[:,1], 'xr', markersize=5)

axs[1].plot(np.reshape(Z,(-1,2))[:,0], np.reshape(Z,(-1,2))[:,1], 'kx')
axs[1].plot(res.F[:,0], res.F[:,1], 'rx')
res
axs[2].plot(res.F[:,0], res.F[:,1], 'x')
plt.show()

In [4]:
def random_acq(GP: GaussianProcess, **kwargs):
    while True:
        x_rand = np.random.uniform(GP.lowerBounds, GP.upperBounds, GP.d)
        if GP.X is None or not x_rand in GP.X:
            break
    return x_rand

## N experiments

In [6]:
savename = "MOONNHeart"

### GPs Initialization
GP = GaussianProcess(O, C, d, lowerBounds, upperBounds, noise_variance=2e-6)

#### Initial samples, at least 1
for l in range(initial_iter):
    ## Get random evaluation point
    while True:
        x_rand = np.random.uniform(lowerBounds[0], upperBounds[0], d)
        if GP.X is None or not x_rand in GP.X:
            break
    ## EVALUATION OF THE OUTSIDE FUNCTION
    
    y_rand = evaluation(x_rand)
    GP.X = np.array([x_rand], dtype=float)
    GP.Y = np.array([y_rand], dtype=float)

GP.updateGP()
GP.optimizeKernel()

print(GP.X)
if False:
    GP.plotSamples()

for l in range(total_iter):

    ## For all acq functions but usemo
    x_best, acq_best = simulated_mes_bins(GP)
    ## For usemo
    #x_best, acq_best = usemo_acq(GP, function = "ei", showplots = False)

    ## EVALUATION OF THE OUTSIDE FUNCTION
    y_best = evaluation(x_best)

    ## UPDATE
    GP.addSample(x_best,y_best)     ## Add new sample to the model
    GP.updateGP()                   ## Update data on the GP regressor
    GP.optimizeKernel()             ## Optimize kernel hyperparameters

    print("Iteration=", l, " x_best=", x_best, "y=", y_best)
    

C = [389.60710364 793.8077877  533.60597056 572.36411548 926.34067191]
Epoch 1/6
1210/1210 [==============================] - 7s 5ms/step - loss: 0.5719 - val_loss: 0.4945
Epoch 2/6
1210/1210 [==============================] - 5s 4ms/step - loss: 0.5242 - val_loss: 0.5037
Epoch 3/6
1210/1210 [==============================] - 4s 3ms/step - loss: 0.5282 - val_loss: 0.5011
Epoch 4/6
1210/1210 [==============================] - 5s 4ms/step - loss: 0.5222 - val_loss: 0.4878
Epoch 5/6
1210/1210 [==============================] - 5s 4ms/step - loss: 0.5276 - val_loss: 0.4876
Epoch 6/6
135/135 [==============================] - 0s 1ms/step
-0.715047619047619 -0.8726173872617388
[[389.60710364 793.8077877  533.60597056 572.36411548 926.34067191]]
C = [182.81494141 214.23583984  63.41552734 786.57958984 130.12451172]
Epoch 1/6
1210/1210 [==============================] - 3s 2ms/step - loss: 0.5374 - val_loss: 0.4922
Epoch 2/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.50

1210/1210 [==============================] - 7s 5ms/step - loss: 0.5438 - val_loss: 0.4773
Epoch 2/6
1210/1210 [==============================] - 6s 5ms/step - loss: 0.5121 - val_loss: 0.4895
Epoch 3/6
1210/1210 [==============================] - 6s 5ms/step - loss: 0.5058 - val_loss: 0.4806
Epoch 4/6
1210/1210 [==============================] - 5s 4ms/step - loss: 0.5044 - val_loss: 0.4783
Epoch 5/6
1210/1210 [==============================] - 4s 4ms/step - loss: 0.5049 - val_loss: 0.4743
Epoch 6/6
135/135 [==============================] - 0s 1ms/step
-0.7715712988192552 -0.7898651789865179
Current x =  [[389.60710364 793.8077877  533.60597056 572.36411548 926.34067191]
 [182.81494141 214.23583984  63.41552734 786.57958984 130.12451172]
 [622.46582031 993.71582031 971.47949219 511.28417969 391.40136719]
 [ 31.02783203 642.52685547 590.31982422 648.32763672 687.96630859]
 [479.37988281 533.52050781 898.00292969 491.94824219 727.84667969]
 [ 89.51904297 305.59814453 331.70166016 111.27

Epoch 4/6
1210/1210 [==============================] - 4s 3ms/step - loss: 0.5285 - val_loss: 0.5272
Epoch 5/6
1210/1210 [==============================] - 4s 4ms/step - loss: 0.5294 - val_loss: 0.4954
Epoch 6/6
135/135 [==============================] - 0s 949us/step
-0.7409983633387889 -0.8419339841933984
Current x =  [[389.60710364 793.8077877  533.60597056 572.36411548 926.34067191]
 [182.81494141 214.23583984  63.41552734 786.57958984 130.12451172]
 [622.46582031 993.71582031 971.47949219 511.28417969 391.40136719]
 [ 31.02783203 642.52685547 590.31982422 648.32763672 687.96630859]
 [479.37988281 533.52050781 898.00292969 491.94824219 727.84667969]
 [ 89.51904297 305.59814453 331.70166016 111.27197266 839.27001953]
 [710.92773438 148.25195312 873.34960938 946.82617188 925.55664062]
 [539.07958984 724.22119141 301.73095703 642.04345703 441.91650391]
 [548.02246094 990.81542969 287.95410156  86.86035156 684.34082031]
 [528.92822266 715.03662109 424.99755859 169.76318359 897.76123047

Iteration= 15  x_best= [551.88964844 399.13574219 485.18066406 315.02441406  92.66113281] y= [-0.78303792]
C = [253.14941406 376.89941406 443.60839844 234.78027344 111.03027344]
Epoch 1/6
1210/1210 [==============================] - 3s 2ms/step - loss: 0.5395 - val_loss: 0.4849
Epoch 2/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5025 - val_loss: 0.4741
Epoch 3/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.4984 - val_loss: 0.4785
Epoch 4/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.4965 - val_loss: 0.4754
Epoch 5/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.4942 - val_loss: 0.4714
Epoch 6/6
135/135 [==============================] - 0s 677us/step
-0.7099598686610726 -0.904695490469549
Current x =  [[389.60710364 793.8077877  533.60597056 572.36411548 926.34067191]
 [182.81494141 214.23583984  63.41552734 786.57958984 130.12451172]
 [622.46582031 993.71582031 971.47949219 511.28417969 391.4013

Iteration= 19  x_best= [452.06787109  49.39697266 868.27392578 825.73486328 896.31103516] y= [-0.78311437]
C = [575.09277344 484.21386719 817.75878906 910.57128906 425.23925781]
Epoch 1/6
1210/1210 [==============================] - 6s 4ms/step - loss: 0.5607 - val_loss: 0.4854
Epoch 2/6
1210/1210 [==============================] - 5s 4ms/step - loss: 0.5251 - val_loss: 0.4929
Epoch 3/6
1210/1210 [==============================] - 3s 3ms/step - loss: 0.5220 - val_loss: 0.4900
Epoch 4/6
1210/1210 [==============================] - 3s 3ms/step - loss: 0.5183 - val_loss: 0.5080
Epoch 5/6
1210/1210 [==============================] - 3s 3ms/step - loss: 0.5199 - val_loss: 0.4967
Epoch 6/6
135/135 [==============================] - 0s 747us/step
-0.7283078131134668 -0.8623895862389587
Current x =  [[389.60710364 793.8077877  533.60597056 572.36411548 926.34067191]
 [182.81494141 214.23583984  63.41552734 786.57958984 130.12451172]
 [622.46582031 993.71582031 971.47949219 511.28417969 391.401

...

Iteration= 26  x_best= [667.66357422 949.48486328 449.65087891  39.72900391 818.00048828] y= [-0.76805721]
C = [925.79833984  90.00244141 131.57470703 193.44970703 271.76025391]
Epoch 1/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5249 - val_loss: 0.4833
Epoch 2/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5036 - val_loss: 0.5003
Epoch 3/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.5008 - val_loss: 0.4792
Epoch 4/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.4986 - val_loss: 0.4740
Epoch 5/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.4947 - val_loss: 0.4810
Epoch 6/6
135/135 [==============================] - 0s 656us/step
-0.7479235880398671 -0.8372849837284984
Current x =  [[389.60710364 793.8077877  533.60597056 572.36411548 926.34067191]
 [182.81494141 214.23583984  63.41552734 786.57958984 130.12451172]
 [622.46582031 993.71582031 971.47949219 511.28417969 391.401

Iteration= 29  x_best= [936.19140625 448.92578125 619.08203125 127.94921875 982.59765625] y= [-0.76915866]
C = [294.72167969 594.42871094 912.50488281 916.37207031 802.29003906]
Epoch 1/6
1210/1210 [==============================] - 3s 2ms/step - loss: 0.5788 - val_loss: 0.4950
Epoch 2/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5498 - val_loss: 0.5115
Epoch 3/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5437 - val_loss: 0.5178
Epoch 4/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5450 - val_loss: 0.5322
Epoch 5/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5404 - val_loss: 0.5095
Epoch 6/6
135/135 [==============================] - 0s 712us/step
-0.7543402777777778 -0.807996280799628
Current x =  [[389.60710364 793.8077877  533.60597056 572.36411548 926.34067191]
 [182.81494141 214.23583984  63.41552734 786.57958984 130.12451172]
 [622.46582031 993.71582031 971.47949219 511.28417969 391.4013

Iteration= 32  x_best= [676.36474609 335.56884766 721.32080078 214.71923828 362.63916016] y= [-0.79007062]
C = [123.35693359 486.38916016 168.31298828 597.57080078 776.42822266]
Epoch 1/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.5387 - val_loss: 0.4808
Epoch 2/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.5084 - val_loss: 0.5258
Epoch 3/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.5081 - val_loss: 0.4825
Epoch 4/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.5095 - val_loss: 0.5065
Epoch 5/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.5081 - val_loss: 0.4740
Epoch 6/6
135/135 [==============================] - 0s 645us/step
-0.7465192465192465 -0.8475127847512784
Current x =  [[389.60710364 793.8077877  533.60597056 572.36411548 926.34067191]
 [182.81494141 214.23583984  63.41552734 786.57958984 130.12451172]
 [622.46582031 993.71582031 971.47949219 511.28417969 391.401

Iteration= 35  x_best= [ 44.32128906 415.57128906 327.59277344 273.45214844 737.51464844] y= [-0.74551266]
C = [314.05761719  64.62402344 831.29394531 719.14550781 473.57910156]
Epoch 1/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.5516 - val_loss: 0.4886
Epoch 2/6
1210/1210 [==============================] - 1s 1ms/step - loss: 0.5242 - val_loss: 0.4924
Epoch 3/6
1210/1210 [==============================] - 1s 1ms/step - loss: 0.5185 - val_loss: 0.4893
Epoch 4/6
1210/1210 [==============================] - 1s 1ms/step - loss: 0.5182 - val_loss: 0.4850
Epoch 5/6
1210/1210 [==============================] - 1s 1ms/step - loss: 0.5163 - val_loss: 0.4942
Epoch 6/6
135/135 [==============================] - 0s 602us/step
-0.7090239410681399 -0.8949325894932589
Current x =  [[389.60710364 793.8077877  533.60597056 572.36411548 926.34067191]
 [182.81494141 214.23583984  63.41552734 786.57958984 130.12451172]
 [622.46582031 993.71582031 971.47949219 511.28417969 391.401

Iteration= 38  x_best= [373.27392578 609.65576172 901.62841797 839.75341797 999.27490234] y= [-0.4102195]
C = [868.99902344 994.68261719  59.79003906 740.41503906 626.33300781]
Epoch 1/6
1210/1210 [==============================] - 9s 7ms/step - loss: 0.5501 - val_loss: 0.4775
Epoch 2/6
1210/1210 [==============================] - 9s 7ms/step - loss: 0.5060 - val_loss: 0.5043
Epoch 3/6
1210/1210 [==============================] - 9s 7ms/step - loss: 0.5029 - val_loss: 0.4740
Epoch 4/6
1210/1210 [==============================] - 9s 7ms/step - loss: 0.5029 - val_loss: 0.4813
Epoch 5/6
1210/1210 [==============================] - 9s 7ms/step - loss: 0.5030 - val_loss: 0.4731
Epoch 6/6
135/135 [==============================] - 0s 1ms/step
-0.7210988172453262 -0.8786610878661087
Current x =  [[389.60710364 793.8077877  533.60597056 572.36411548 926.34067191]
 [182.81494141 214.23583984  63.41552734 786.57958984 130.12451172]
 [622.46582031 993.71582031 971.47949219 511.28417969 391.401367

## Evaluate results

As you are doing a real world experiment you do not have access to the real pareto front, then you fo noy hsbr yhr metrics as before (I removed them).

What you should evaluate is the results in terms of the best *y* possible. You have the data below, but you may do a pandas dataframe to store it cleaner when it is computed in the previous section.

GP.X === samples where the function was evaluated

GP.Y === results of those evaluations

In [ ]:
plt.scatter(-GP.Y[:,0],-GP.Y[:,1])

In [ ]:
from utils.calc_pareto import get_pareto_undominated_by

front = get_pareto_undominated_by(-GP.Y)

plt.scatter(front[:,0],front[:,1])

In [ ]:
-GP.Y

In [ ]:
x_front = [GP.X[(-GP.Y).tolist().index(y)] for y in front.tolist()]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

front_test = []

data = pd.read_csv("heart_disease_health_indicators_BRFSS2015.csv")
X = data.drop("HeartDiseaseorAttack", axis=1)
y = data["HeartDiseaseorAttack"]

y_1 = y[y == 1]
y_used = pd.concat([y_1, y[y == 0].sample(len(y_1))])
X_used = X.iloc[y_used.index]

X_train_val, X_test, y_train_val, y_test = train_test_split(X_used, y_used, test_size=0.1, random_state=42, stratify=y_used)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42, stratify=y_train_val)

checkpoint_path = f"./Checkpoints"
checkpoint_dir = [f for f in os.listdir(checkpoint_path) if os.path.isfile(os.path.join(checkpoint_path, f)) and f.split(".")[-1]=="h5"]


for path in checkpoint_dir:
    print(path.split(".")[0].split("/")[-1].split("_")[-2:])
    arch = [int(x) for x in path.split(".")[0].split("/")[-1].split("_")[-2:]]
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(21, 1)),
      tf.keras.layers.Dense(int(np.around(arch[0])), activation='relu'),
      tf.keras.layers.Dense(int(np.around(arch[1])), activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    

    
    model.load_weights(path)
    
    y_pred = np.around(model.predict(X_test))
    
    front_test.append((precision_score(y_test, y_pred), recall_score(y_test, y_pred)))
    

In [ ]:
[x[0] for x in front_test]

plt.scatter([x[0] for x in front_test], [x[1] for x in front_test])

In [ ]:
front_test

In [ ]:
#!rm -rf Checkpoints